# Website

> Throughout this analysis, we will extract data from websites. We need to ensure that we do so responsibly by sending a user agent and caching as much data as possible. We create classes and methods to make that possible here.

In [ ]:
# | default_exp website
# | export
from requests_cache import SQLiteCache, CachedSession
from pydantic import BaseModel
from pathlib import Path
from typing import Optional

In [ ]:
# | export
class Website:
    """
    Represents a Website. You can make cached requests to the website by
    using the `self.session` object.
    """

    def __init__(
        self,
        cache_name: str,
        cache_dir: Path,
        cache_methods: tuple = ("GET", "HEAD", "POST"),
        user_agent: str = "legisph (code@tjpalanca.com)",
    ):
        cache_dir.mkdir(exist_ok=True, parents=True)
        self.session = CachedSession(
            backend=SQLiteCache(cache_dir / f"{cache_name}.sqlite"),
            allowable_methods=cache_methods,
        )
        self.session.headers["User-Agent"] = user_agent

In cases where we are unable to successfully parse, we create custom exceptions to know whether this is something we can act on or if it is something we cannot solve.

In [ ]:
# | export
class WebsiteException(Exception):
    """
    General class for exceptions from a website. Has a resource string that can be
    accessed to know about the problem.
    """

    def __init__(self, message: str, resource: str):
        super().__init__(message, resource)

    # We have to do this instead of using instance fields because it is not possible
    # to unpickle this exception with fields.
    @property
    def message(self):
        return self.args[0]

    @property
    def resource(self):
        return self.args[1]


class NotFoundError(WebsiteException):
    """Raised when the resource is not found."""


class ServerError(WebsiteException):
    """Raised when the server reports an internal error."""


class ParsingError(WebsiteException):
    """Raised when there are issues parsing the exception"""

Some objects are common across different website and we define them below.

In [ ]:
# | export
class Link(BaseModel):
    """A link to a resource on the internet"""

    url: str
    name: Optional[str]